# 🚀 Fusion RAG: Enhanced Retrieval-Augmented Generation

Welcome to the **Fusion RAG** project! This notebook demonstrates an advanced Retrieval-Augmented Generation (RAG) pipeline inspired by the [RAG-Fusion](https://arxiv.org/pdf/2402.03367) approach. The workflow leverages multiple query reformulations, document retrieval, and robust re-ranking to generate high-quality, contextually rich responses using Large Language Models (LLMs).

---

## 🧩 How It Works

1. **Query Reformulation**  
    For a user's input query, we generate _n_ semantically diverse sub-queries to cover different aspects of the information need.

2. **Document Retrieval**  
    For each sub-query, we retrieve _m_ relevant documents from the knowledge base.

3. **Re-ranking with RRF**  
    All retrieved documents are re-ranked using the Reciprocal Rank Fusion (RRF) algorithm, which aggregates scores across queries to prioritize the most relevant content.

4. **Fusion and Response Generation**  
    The top fused documents and the set of sub-queries are provided to the LLM, which synthesizes a detailed and factual answer.

---

## 🛠️ Features

- **Multi-query Expansion:** Improves recall by exploring diverse phrasings of the user's question.
- **RRF-based Fusion:** Robustly combines evidence from multiple retrievals ([see paper](https://arxiv.org/pdf/2402.03367)).
- **LLM-powered Synthesis:** Generates comprehensive answers grounded in retrieved evidence.

---

## 📦 Example Pipeline

```python
user_query = "What is Amazon AWS?"
response = Fusion_RAG_Pipeline(user_query)
print(response)
```

---

## 📚 Reference

- **RAG-FUSION: A NEW TAKE ON RETRIEVAL-AUGMENTED GENERATION**  
  [arXiv:2402.03367](https://arxiv.org/pdf/2402.03367)

---

## ✨ Why Use Fusion RAG?

- 🔍 **Better Coverage:** Multiple queries ensure broader information retrieval.
- 🏆 **Improved Ranking:** RRF fusion surfaces the most relevant documents.
- 🤖 **LLM Integration:** Final answers are both accurate and contextually rich.

---

## 🚦 Get Started

1. Prepare your document corpus in the `data/` directory.
2. Run the notebook cells in order.
3. Use `Fusion_RAG_Pipeline(your_query)` to get high-quality, fused responses!

---

Happy experimenting! 💡

In [3]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [5]:
# %pip install -U langchain-google-genai
# %pip install -qU langchain-groq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
llm_2 = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

In [ ]:
!pip install llama_index.llms.langchain
!pip install langchain_community
%pip install llama-index-embeddings-google-genai
!pip install llama_index

In [6]:
from llama_index.core import VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
documents = SimpleDirectoryReader("data").load_data()
splitter = SentenceSplitter(chunk_size=1000,chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(documents)

embed_model = GoogleGenAIEmbedding(
    model_name="text-embedding-004",
    embed_batch_size=100,
)
Settings.embed_model = embed_model
Settings.llm = llm
vector_index = VectorStoreIndex(nodes)
retriever = vector_index.as_retriever(similarity_top_k=5)

In [7]:
def generate_multiple_queries(query, n=4):
    """
    For a given query, generate n semantically diverse but relevant sub-queries.
    """
    prompt = f"""
You are an expert in reformulating user queries for improved document retrieval.

Given the original query:
"{query}"

Generate {n} semantically diverse sub-queries that explore different ways of asking about the same topic.
Return the sub-queries as a numbered list.
"""
    response = llm.invoke(prompt)
    return [line.split(". ", 1)[1] for line in response.content.strip().split("\n") if ". " in line]

In [18]:
multiple_queries= generate_multiple_queries("what is Amazon AWS?")

In [20]:
def retrieve_docs_for_each_query(queries,top_k=5):
  list_of_all_docs=[]
  """
  Retrieve documents for each query.
  """
  for _query in queries:
    list_of_all_docs.append(retriever.retrieve(_query))
  return list_of_all_docs

In [21]:
list_of_all_docs=retrieve_docs_for_each_query(multiple_queries)

In [ ]:
from collections import defaultdict

def ReRankWithRRF(list_of_all_docs, k=60, final_top_k=7):
    """
    Re-rank all retrieved documents across queries using RRF.
    Returns a fused, re-ranked list of top documents.
    """
    doc_scores = defaultdict(float)
    doc_ids = {}

    for doc_list in list_of_all_docs:
        for rank, doc in enumerate(doc_list):
            # Use doc ID or content as unique key
            key = getattr(doc, "id", doc.metadata.get("source", None)) or doc.text.strip()
            rrf_score = 1 / (k + rank + 1)
            doc_scores[key] += rrf_score
            doc_ids[key] = doc  # Keep the original doc

    # Sort by aggregated RRF scores
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    for doc_key, score in sorted_docs[:final_top_k]:
     doc_ids[doc_key].score = score
    # Return top final_top_k documents
    return [doc_ids[doc_key] for doc_key, _ in sorted_docs[:final_top_k]]

In [42]:
fused_docs = ReRankWithRRF(list_of_all_docs)

In [ ]:
def generate_fusion_rag_response(original_query, sub_queries, fused_docs, llm):
    """
    Generate a final response using the original query, sub-queries, and fused documents.
    """

    sub_query_section = "\n".join([f"{i+1}. {q}" for i, q in enumerate(sub_queries)])

    doc_section = "\n\n".join([
        f"Document {i+1}:\n{doc.text.strip()}"
        for i, doc in enumerate(fused_docs)
    ])

    prompt = f"""
You are a helpful assistant.

A user asked the question:
"{original_query}"

To improve retrieval, we reformulated it into the following sub-queries:
{sub_query_section}

We retrieved the following relevant documents:
{doc_section}

Based on the documents above, answer the user's original question in a detailed and factual manner.
"""

    # Call the LLM
    response = llm.invoke(prompt)
    return response.content.strip()

In [ ]:
user_query="what is amazon AWS ?"
response = generate_fusion_rag_response(
    original_query=user_query,
    sub_queries=multiple_queries,
    fused_docs=fused_docs,
    llm=llm_2
)
print(response)

Amazon Web Services (AWS) is a comprehensive cloud computing platform provided by Amazon that offers a wide array of services to support businesses in building, deploying, and managing applications and workloads. Launched in 2006, AWS introduced a pay-as-you-go model, allowing businesses to replace upfront capital expenses with variable costs that scale according to their needs. This model eliminates the need for planning and procuring physical infrastructure, enabling quick resource provisioning.

AWS provides over 140 services, including compute, storage, databases, analytics, networking, mobile, developer tools, IoT, security, and enterprise applications. These services are available on-demand, with no long-term commitments, making AWS accessible to businesses of all sizes, from startups to large enterprises. The platform emphasizes flexibility, scalability, and cost-efficiency, allowing businesses to respond swiftly to changing demands without managing physical servers.

With a glo

In [62]:
def Fusion_RAG_Pipeline(query):
  multiple_queries= generate_multiple_queries(query)
  list_of_all_docs=retrieve_docs_for_each_query(multiple_queries)
  fused_docs = ReRankWithRRF(list_of_all_docs)
  response = generate_fusion_rag_response(
    original_query=query,
    sub_queries=multiple_queries,
    fused_docs=fused_docs,
    llm=llm_2
  )
  return response

In [52]:
Settings.llm=llm_2

In [53]:
query_2="what are some Amazon Web Services Cloud Platforms ?"
response_2=Fusion_RAG_Pipeline(query_2)
print(response_2)

Amazon Web Services (AWS) is a comprehensive cloud computing platform that offers a wide array of services designed to meet various business needs. Here's an organized overview of AWS cloud platforms:

### What is AWS?
AWS is a subsidiary of Amazon that provides on-demand cloud computing services and APIs to individuals, businesses, and governments. It offers a highly reliable, scalable, and low-cost infrastructure platform that supports a broad range of applications, from simple websites to complex enterprise applications.

### Main Categories of AWS Services

1. **Compute Services**
   - **Amazon EC2 (Elastic Compute Cloud):** Provides virtual servers for computing and storage, allowing users to run applications in the cloud.
   - **Amazon Lambda:** A serverless compute service that runs code in response to events without requiring server management.
   - **Amazon Elastic Container Service (ECS):** A container orchestration service for Docker containers.

2. **Storage Services**
   -

In [49]:
query_3="how AWS relates to IoT(internet of things) ?"
response_3=Fusion_RAG_Pipeline(query_3)
print(response_3)

AWS provides a comprehensive suite of services and tools that cater to the needs of the Internet of Things (IoT), enabling businesses to build, manage, and scale IoT solutions effectively. Here's a detailed overview of how AWS relates to IoT:

### AWS IoT Services

1. **AWS IoT Core**: This service is designed for device connectivity and management. It allows secure communication between IoT devices and the cloud, enabling efficient data exchange and device management.

2. **AWS IoT Greengrass**: Focused on edge computing, this service allows data processing at the edge, reducing latency and enabling devices to operate even without a constant internet connection.

3. **AWS IoT Analytics**: This fully managed service is tailored for processing and analyzing IoT data. It supports ad-hoc and scheduled queries, machine learning integration, and can handle large volumes of data from millions of devices.

4. **AWS IoT Device Management**: This service simplifies managing large fleets of IoT 

In [54]:
#NAIVE RAG RESPONSE NOT THAT GOOD AS COMPARED TO FUSION RAG
query_engine=vector_index.as_query_engine()
response_3=query_engine.query(query_3)
print(response_3)

AWS offers a comprehensive suite of services that simplify and enhance IoT (Internet of Things) solutions. These services include:

1. **AWS IoT 1-Click**: Enables simple devices to trigger AWS Lambda functions, allowing actions like notifications or asset tracking without requiring firmware changes or complex configurations.

2. **AWS IoT Analytics**: A fully-managed service that processes large volumes of IoT data, automating filtering, transformation, and enrichment to make data analysis more effective, especially when integrated with third-party data.

3. **AWS IoT SiteWise**: Facilitates industrial data collection and management by connecting to on-premises equipment, enabling performance monitoring and application development to optimize operations.

4. **AWS IoT Things Graph**: Provides a visual interface to connect devices and services, simplifying the creation of IoT applications through a drag-and-drop model, thus reducing the need for extensive coding.

These services collec

In [50]:
query_4="tell about media services of Amazon AWS"
response_4=Fusion_RAG_Pipeline(query_4)
print(response_4)

Amazon Web Services (AWS) offers a comprehensive suite of media services designed to support the media and entertainment industry with scalable, flexible, and cost-effective solutions. These services cater to various aspects of media workflows, including content creation, processing, delivery, and monetization. Below is a detailed overview of AWS media services, focusing on their core offerings, solutions for video streaming and live broadcasting, and support for content workflows.

### Core Media and Entertainment Services Offered by AWS

AWS provides a range of services tailored to meet the needs of media professionals, from live video streaming to content delivery and monetization. These services are designed to handle the complexities of modern media workflows, ensuring high performance, reliability, and security.

1. **AWS Elemental MediaConnect**  
   AWS Elemental MediaConnect is a cloud-based service that enables the transport of live video content. It allows users to ingest li

In [56]:
query_5="what are the storage services provided by amazon AWS ?"
response_5=Fusion_RAG_Pipeline(query_5)
print(response_5)

Amazon Web Services (AWS) offers a comprehensive suite of storage services designed to meet various data management needs. Here is an organized overview of the storage services mentioned in the provided documents:

1. **Amazon S3 (Simple Storage Service)**:
   - **Purpose**: Object storage for data such as media, documents, and other files.
   - **Features**: High durability (11 nines), scalable, and accessible via REST API or SDKs.

2. **Amazon Elastic Block Store (EBS)**:
   - **Purpose**: Block-level storage for EC2 instances.
   - **Features**: Persistent, automatically replicated within an Availability Zone for high availability and low-latency performance.

3. **Amazon Elastic File System (EFS)**:
   - **Purpose**: Fully managed file storage service for distributed or shared file systems.
   - **Features**: Supports multiple EC2 instances accessing shared files simultaneously.

4. **Amazon FSx**:
   - **Purpose**: Fully managed file system service, including support for Windows-c

In [60]:
query_6="what are the security services provided by amazon AWS"
response_6=Fusion_RAG_Pipeline(query_6)
print(response_6)

Amazon Web Services (AWS) provides a comprehensive suite of security services designed to protect data, ensure compliance, and offer scalable solutions. Here is an organized overview of the key security services offered by AWS:

1. **Infrastructure Security**: AWS ensures a secure data center and network architecture, allowing customers to benefit from a high level of security without managing physical servers, thus reducing costs.

2. **Compliance and Governance**: AWS manages numerous compliance programs, simplifying the compliance process for customers by handling a significant portion of compliance requirements.

3. **AWS Single Sign-On (SSO)**: This service enables centralized management of access to multiple AWS accounts and business applications, supporting SAML integrations with various third-party apps.

4. **Amazon GuardDuty**: A threat detection service that uses threat intelligence and machine learning to identify unauthorized activities and potential threats, providing det

In [63]:
query_7="what are services AWS provides related to Machine Learning or AI "
response_7=Fusion_RAG_Pipeline(query_7)
print(response_7)

AWS offers a comprehensive suite of services tailored for Machine Learning (ML) and Artificial Intelligence (AI). These services cater to various aspects of the ML lifecycle, from data preparation and model training to deployment and inference. Here's an organized overview of the key services:

1. **Amazon SageMaker**: A fully managed platform designed to simplify the entire ML workflow. It supports building, training, and deploying models, integrating seamlessly with other AWS services.

2. **Amazon Personalize**: Specializes in creating personalized recommendations by analyzing data and training custom models, ideal for enhancing user experiences with tailored suggestions.

3. **AWS Deep Learning AMIs**: Pre-configured Amazon EC2 instances that come with popular deep learning frameworks, allowing users to quickly set up environments for model training and experimentation.

4. **AWS Inferentia**: A custom-built ASIC chip optimized for high-performance inference, offering cost-effectiv

In [65]:
query_8="what are the Management and Governance services of Amazon AWS ?"
response_8=Fusion_RAG_Pipeline(query_8)
print(response_8)

**Management and Governance Services of Amazon AWS**

Amazon Web Services (AWS) offers a comprehensive suite of tools and services designed to help users effectively manage their cloud resources and ensure governance. These services can be categorized into Management and Governance, each playing a crucial role in maintaining efficiency, security, and compliance in the cloud environment.

### Management Services

1. **AWS Identity and Access Management (IAM)**
   - **Description:** IAM allows users to securely control access to AWS resources by creating and managing users, groups, and permissions. It ensures that only authorized personnel have access to specific resources, enhancing security and governance.

2. **AWS CloudTrail**
   - **Description:** CloudTrail records API calls and delivers log files, providing a history of API activity. This service is essential for auditing, tracking changes, and ensuring compliance.

3. **AWS Config**
   - **Description:** Config offers resource in